In [1]:
from recordclass import dataobject, clsconfig, litelist
import matplotlib.pyplot as plt
import numpy as np
import time
import gc
from random import randint

In [2]:
class Point(dataobject):
    x:int
    y:int
    z:int
    __options__ = {'fast_new':True}

@clsconfig(gc=True)
class Point_GC(dataobject):
    x:int
    y:int
    z:int
    __options__ = {'fast_new':True}
        
class Point_Slots:
    __slots__ = 'x','y'
    x:int
    y:int
    z:int
        
    def __init__(self, x, y):
        self.x = x; self.y = y; self.z = z

In [3]:
N = 100000
def test_dataobjects(N, randint=randint, Point=Point):
    lst = [Point(randint(0, 10), randint(0, 10), randint(0, 10)) for i in range(N)]
    for i in range(10*N):
        j = randint(0,N-1)
        lst[j] = Point(randint(0, 10), randint(0, 10), randint(0, 10))
        
def test_dataobjects_gc(N, randint=randint, Point_GC=Point_GC):
    lst = [Point_GC(randint(0, 10), randint(0, 10), randint(0, 10)) for i in range(N)]
    for i in range(10*N):
        j = randint(0,N-1)
        lst[j] = Point_GC(randint(0, 10), randint(0, 10), randint(0, 10))


In [4]:
gc.collect()
%timeit test_dataobjects(N)

2.61 s ± 6.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
gc.collect()
%timeit test_dataobjects_gc(N)

2.67 s ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
(3.36-3.29)/3.29*100

2.12765957446808